In [9]:
import pymysql
import json
from functools import partial
import csv
import networkx as nx
import freeman as fm
from random import random, choices
def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro

def multiple_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        res = cursor.fetchall()
        return res

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='vlm1998',
    database='redes')

db = partial(run_db_query, connection)

In [10]:
def add_node(g, n):
    g.add_node(n)
    g.nodes[n]['pos'] = (random(), random())
    g.nodes[n]['label'] = n
    g.nodes[n]['color'] = (255, 0, 0)

def add_edge(g, n, m, peso):
    print("De: ", n, "para: ", m, "peso: ", peso)
    g.add_edge(n, m)
    g[n][m]['weight'] = peso

def get_team_name(connection, id_team):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times WHERE id_time = %s", (id_team))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def get_team_id(connection, nome):
    with connection.cursor() as cursor:
        cursor.execute("SELECT id_time FROM times WHERE nome=%s", (nome))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def list_all_teams(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times")
        res = cursor.fetchall()
        return res
def list_all_transfers(connection, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM transferencia WHERE ano=%s", (ano))
        res = cursor.fetchall()
        return res
def get_from_where(connection, id_jogador, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT times.nome FROM waived INNER JOIN times USING(id_time) WHERE id_jogador = %s AND ano = %s ", (id_jogador, str(ano)))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1

def get_weight(connection, id_time1, id_time2, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT peso FROM rivalidade WHERE id_time1=%s AND id_time2=%s AND ano=%s", (id_time1, id_time2, ano))
        res = cursor.fetchall()
        if res:
            return res
        else: #Pode ser que os nomes estejam invertidos nao sei
            cursor.execute("SELECT peso FROM rivalidade WHERE id_time1=%s AND id_time2=%s AND ano=%s", (id_time2, id_time1, ano))
            if res:
                return res
            return -1


In [13]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2019)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2019)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    #else:
        #print(antigo, novo)
    
    
a.rec(g)
a.play()

De:  ('Miami Heat',) para:  ('Houston Rockets',) peso:  0
De:  ('Miami Heat',) para:  ('Houston Rockets',) peso:  0
De:  ('Miami Heat',) para:  ('Houston Rockets',) peso:  0
De:  ('Miami Heat',) para:  ('Houston Rockets',) peso:  0
De:  ('Miami Heat',) para:  ('Houston Rockets',) peso:  0
De:  ('Miami Heat',) para:  ('Houston Rockets',) peso:  0
De:  ('Atlanta Hawks',) para:  ('Houston Rockets',) peso:  0
De:  ('Atlanta Hawks',) para:  ('Houston Rockets',) peso:  0
De:  ('Atlanta Hawks',) para:  ('Houston Rockets',) peso:  0
De:  ('Atlanta Hawks',) para:  ('Houston Rockets',) peso:  0
De:  ('Atlanta Hawks',) para:  ('Houston Rockets',) peso:  0
De:  ('Atlanta Hawks',) para:  ('Houston Rockets',) peso:  0
De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('New York Knicks

De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('New York Knicks',) para:  ('Washington Wizards',) peso:  0
De:  ('Utah Jazz',) para:  ('Golden State Warriors',) peso:  0
De:  ('Orlando Magic',) para:  ('Atlanta Hawks',) peso:  0
De:  ('Orlando Magic',) para:  ('Atlanta Hawks',) peso:  0
De:  ('Orlando Magic',) para:  ('Atlanta Hawks',) peso:  0
De:  ('Orlando Magic',) para:  ('Atlanta Hawks',) peso:  0
De:  ('Orlando Magic',) para:  ('Atlanta Hawks',) peso:  0
De:  ('Orlando Magic',) para:  ('Atlanta Hawks',) peso:  0
De:  ('Detroit Pistons',) para:  ('Houston Rockets',) peso:  0
De:  ('Washington Wizards',) para:  ('Los Angeles Lakers',) peso:  0
De:  ('Washington Wizards',) para:  ('Los Angeles Lakers',) peso:  0
De:  ('Washington Wizards',) para:  ('Los Angeles La

De:  ('Sacramento Kings',) para:  ('Minnesota Timberwolves',) peso:  0
De:  ('Sacramento Kings',) para:  ('Minnesota Timberwolves',) peso:  0
De:  ('Sacramento Kings',) para:  ('Minnesota Timberwolves',) peso:  0
De:  ('Sacramento Kings',) para:  ('Minnesota Timberwolves',) peso:  0
De:  ('Sacramento Kings',) para:  ('Minnesota Timberwolves',) peso:  0
De:  ('Sacramento Kings',) para:  ('Minnesota Timberwolves',) peso:  0
De:  ('Phoenix Suns',) para:  ('Houston Rockets',) peso:  0
De:  ('Phoenix Suns',) para:  ('Houston Rockets',) peso:  0
De:  ('Phoenix Suns',) para:  ('Houston Rockets',) peso:  0
De:  ('Phoenix Suns',) para:  ('Houston Rockets',) peso:  0
De:  ('Phoenix Suns',) para:  ('Houston Rockets',) peso:  0
De:  ('Phoenix Suns',) para:  ('Houston Rockets',) peso:  0
De:  ('Utah Jazz',) para:  ('Detroit Pistons',) peso:  0
De:  ('Utah Jazz',) para:  ('Detroit Pistons',) peso:  0
De:  ('Utah Jazz',) para:  ('Detroit Pistons',) peso:  0
De:  ('Utah Jazz',) para:  ('Detroit Piston

In [7]:
connection.close()

Error: Already closed